# 파이썬으로 엑셀다루기 (with Pandas)
_본 자료는 안수찬 강사님의 파이썬을 활용한 업무자동화 Camp (fast campus)의 강의자료를 기반으로 만들어졌습니다._  
만든이 : 김보섭  

### Filtering example 1

In [1]:
# Filtering
# map, filter를 그대로 활용

In [2]:
numbers = [-100, 100, -200, 200, -300, 300]

In [3]:
list(filter(lambda x : x >= 0, numbers))

# 1. map
# 2. map을 한 결과 중에서 True 만을 뽑는 거

[100, 200, 300]

In [4]:
list(map(lambda x : x >= 0, numbers)) # Filtering Mask, Boolean Mask

[False, True, False, True, False, True]

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(columns = ['name', 'price'])

In [7]:
df.loc[0] = ['apple', 1000]
df.loc[1] = ['banana', 1500]
df.loc[2] = ['kiwi', 3000]

In [8]:
# 2000원 이하의 과일 뽑자. => [True. True, False]
df.price # pd.DataFrame의 한 열, 한 행 => Series

0    1000
1    1500
2    3000
Name: price, dtype: object

In [9]:
df.price <= 2000

0     True
1     True
2    False
Name: price, dtype: bool

In [10]:
df[df.price <= 2000]

,name,price
0,apple,1000
1,banana,1500


In [11]:
df

,name,price
0,apple,1000
1,banana,1500
2,kiwi,3000


In [12]:
# 만약 name에 'a'가 포함되면/시작하면 필터링
# Series에 내장된 기능
df.name.str.contains('a')

0     True
1     True
2    False
Name: name, dtype: bool

In [13]:
is_startwith_a = df.name.str.startswith('a')
df[is_startwith_a]

,name,price
0,apple,1000


### Filtering example 2

In [14]:
df = pd.DataFrame(columns = ['name', 'price', 'count'])
df.loc[0] = ['apple', 1000, 10]
df.loc[1] = ['banana', 1500, 20]
df.loc[2] = ['kiwi', 3000, 5]
df

,name,price,count
0,apple,1000,10
1,banana,1500,20
2,kiwi,3000,5


In [15]:
df['amount'] = df.price * df['count']
df

,name,price,count,amount
0,apple,1000,10,10000
1,banana,1500,20,30000
2,kiwi,3000,5,15000


In [16]:
df.amount.sum()

55000

### 직방 example
직방 API를 활용하여 DataFrame을 구성  
scrapping link : https://api.zigbang.com/v3/items?detail=true&item_ids=[8730048,8606664,8585140,8748326,8656700,8692737,8627072,8672664,8753748,8677208,8696184,8684748,8680502,8605456,8748470,8745372,8703797,8632872,8450371,8747438,8731881,8669325,8668688,8677036,8634676,8751376,8673253,8692952,8751407,8750738,8744394,8735229,8719062,8685405,8757159,8719839,8741145,8325448,8707734,8671689,8702652,8751413,8719934,8039087,8697444,8748097,8704583,8726705,8720767,8728499,8649187,8660350,8731727,8734916,8712824,8709501,8744896,8695530,8598979,8481781]

#### 보증금 1000만원, 월세 70만원 이하의 매물만 뽑기
#### 1. 데이터 생성

In [17]:
# 1. for loop
# 2. preprocess
# ---------------
import requests, json
response = requests.get('https://api.zigbang.com/v3/items?detail=true&item_ids=[8730048,8606664,8585140,8748326,8656700,8692737,8627072,8672664,8753748,8677208,8696184,8684748,8680502,8605456,8748470,8745372,8703797,8632872,8450371,8747438,8731881,8669325,8668688,8677036,8634676,8751376,8673253,8692952,8751407,8750738,8744394,8735229,8719062,8685405,8757159,8719839,8741145,8325448,8707734,8671689,8702652,8751413,8719934,8039087,8697444,8748097,8704583,8726705,8720767,8728499,8649187,8660350,8731727,8734916,8712824,8709501,8744896,8695530,8598979,8481781]')

In [18]:
data = response.json() # 파이썬 Dict

In [19]:
df1 = pd.DataFrame(columns = ['deposit', 'rent'])
# 1. for loop
for item in data.get('items'):
    deposit = item.get('item').get('deposit')
    rent = item.get('item').get('rent')
    #print(deposit, rent)
    
    # df.loc[len(df)] - append new row
    df1.loc[len(df1)] = [deposit, rent]

In [20]:
# 2. list comprehension (dictionary의 list)
df2 = [{'deposit' : item.get('item').get('deposit'), 'rent' : item.get('item').get('rent')} for item in data.get('items')]    

In [21]:
# 3. list comprehension + select column
df3 = pd.DataFrame([item.get('item') for item in data.get('items')])
df3['rent'].head() # Series

0    75
1     0
2    45
3    60
4     0
Name: rent, dtype: int64

In [22]:
df3[['rent']].head() # DataFrame (1Column)

,rent
0,75
1,0
2,45
3,60
4,0


In [23]:
df3[['deposit', 'rent']].head() # DataFrame (2Column)

,deposit,rent
0,2000,75
1,15000,0
2,1000,45
3,4000,60
4,24000,0


In [24]:
rooms_df = df3[['rent', 'deposit', 'id', 'agent_address1']]
rooms_df.head()

,rent,deposit,id,agent_address1
0,75,2000,8730048,서울특별시 송파구 삼전동 121-3 101호
1,0,15000,8606664,서울특별시 중랑구 중화동 283-11
2,45,1000,8585140,서울특별시 광진구 화양동 92-3
3,60,4000,8748326,서울특별시 강남구 봉은사로34길 9 1층(역삼동)
4,0,24000,8656700,서울특별시 광진구 구의동 243-31


#### 2. 보증금 1000만원이하에 월세가 70만원이하인 집만 filtering

In [25]:
# 보증금 1000만원이하에 월세가 70만원이하인 집만 filtering
# Filtering Mask 생성이 관건
is_low_deposit = rooms_df.deposit <= 1000
is_low_rent = rooms_df.rent <= 70

In [26]:
rooms_df[is_low_deposit & is_low_rent].head()

,rent,deposit,id,agent_address1
2,45,1000,8585140,서울특별시 광진구 화양동 92-3
6,55,500,8627072,서울특별시 광진구 화양동 8-57 1층
8,60,500,8753748,서울특별시 중랑구 면목동 616-35
9,60,1000,8677208,서울특별시 강남구 논현로67길 48 1층(역삼동)
13,60,1000,8605456,서울특별시 송파구 백제고분로36가길 9


In [27]:
rooms_df[is_low_deposit][is_low_rent].head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,rent,deposit,id,agent_address1
2,45,1000,8585140,서울특별시 광진구 화양동 92-3
6,55,500,8627072,서울특별시 광진구 화양동 8-57 1층
8,60,500,8753748,서울특별시 중랑구 면목동 616-35
9,60,1000,8677208,서울특별시 강남구 논현로67길 48 1층(역삼동)
13,60,1000,8605456,서울특별시 송파구 백제고분로36가길 9


In [28]:
rooms_df[rooms_df.rent <= 50][rooms_df.deposit <= 15000].head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,rent,deposit,id,agent_address1
1,0,15000,8606664,서울특별시 중랑구 중화동 283-11
2,45,1000,8585140,서울특별시 광진구 화양동 92-3
5,30,3000,8692737,서울특별시 광진구 화양동 36-39
11,40,3000,8684748,서울특별시 성동구 마조로 52 (마장동)
12,0,9000,8680502,서울특별시 동대문구 휘경1동 회기로 184


#### 3. agent_addres1에서 '1층'이라는 텍스트가 포함된 매물 뽑기
1. agent_address1 => '주소'
2. '주소'에서 '1층'이라는 텍스트가 포함된 Row

In [29]:
# 원본 DF를 바꾸는 방법
# 1. 새로운 변수에 할당
# 2. 원본 DF를 가리키던 변수에 다시 할당
# 3. DataFrame 관련 함수들은 옵션으로 'inplace=True'라고 해주면 원본이 변경됨 <- pandas의 경우 대부분 3번

# agent_address1 => '주소'
rooms_df.rename(columns={'agent_address1' : '주소'}, inplace = True)
rooms_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,rent,deposit,id,주소
0,75,2000,8730048,서울특별시 송파구 삼전동 121-3 101호
1,0,15000,8606664,서울특별시 중랑구 중화동 283-11
2,45,1000,8585140,서울특별시 광진구 화양동 92-3
3,60,4000,8748326,서울특별시 강남구 봉은사로34길 9 1층(역삼동)
4,0,24000,8656700,서울특별시 광진구 구의동 243-31


In [30]:
# '주소'에서 '1층'이라는 텍스트가 포함된 Row 추출
rooms_df[(rooms_df.id.apply(lambda x : str(x)) + ' '+rooms_df.주소).str.contains('1층')]

,rent,deposit,id,주소
3,60,4000,8748326,서울특별시 강남구 봉은사로34길 9 1층(역삼동)
6,55,500,8627072,서울특별시 광진구 화양동 8-57 1층
9,60,1000,8677208,서울특별시 강남구 논현로67길 48 1층(역삼동)
10,80,80,8696184,서울특별시 강남구 역삼동 696-11 1층
16,45,500,8703797,"서울특별시 성동구 마조로9길 15, 1층(행당동)"
23,270,270,8677036,서울특별시 강남구 봉은사로30길 42 1층
25,85,85,8751376,서울특별시 강남구 역삼동 696-11번지 지하 1층
32,35,2500,8719062,서울특별시 광진구 화양동 37-17.1층
34,0,23000,8757159,"서울특별시 강남구 봉은사로 47길 12, 1층"
36,0,33000,8741145,서울특별시 강남구 선릉로72길 35 1층(대치동)


#### 4. 전세가격 변수 ('standard_price')를 만들고 내림차순으로 정렬

In [31]:
rooms_df['standard_price'] = rooms_df.deposit + rooms_df.rent * 100
rooms_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,rent,deposit,id,주소,standard_price
0,75,2000,8730048,서울특별시 송파구 삼전동 121-3 101호,9500
1,0,15000,8606664,서울특별시 중랑구 중화동 283-11,15000
2,45,1000,8585140,서울특별시 광진구 화양동 92-3,5500
3,60,4000,8748326,서울특별시 강남구 봉은사로34길 9 1층(역삼동),10000
4,0,24000,8656700,서울특별시 광진구 구의동 243-31,24000


In [32]:
rooms_df.sort_values(by = 'standard_price', ascending = False).head(10)

,rent,deposit,id,주소,standard_price
36,0,33000,8741145,서울특별시 강남구 선릉로72길 35 1층(대치동),33000
52,0,32000,8731727,서울특별시 강남구 선릉로72길 35 1층(대치동),32000
45,0,32000,8748097,서울특별시 광진구 자양동 609-5,32000
23,270,270,8677036,서울특별시 강남구 봉은사로30길 42 1층,27270
39,210,5000,8671689,서울특별시 강남구 역삼동 750-8번지 동진빌딩1층,26000
7,0,25000,8672664,서울특별시 동대문구 이문로 58,25000
4,0,24000,8656700,서울특별시 광진구 구의동 243-31,24000
47,230,230,8726705,서울특별시 강남구 봉은사로30길 42 1층,23230
34,0,23000,8757159,"서울특별시 강남구 봉은사로 47길 12, 1층",23000
31,0,23000,8735229,서울특별시 광진구 구의강변로 52,23000


In [33]:
rooms_df.head()

,rent,deposit,id,주소,standard_price
0,75,2000,8730048,서울특별시 송파구 삼전동 121-3 101호,9500
1,0,15000,8606664,서울특별시 중랑구 중화동 283-11,15000
2,45,1000,8585140,서울특별시 광진구 화양동 92-3,5500
3,60,4000,8748326,서울특별시 강남구 봉은사로34길 9 1층(역삼동),10000
4,0,24000,8656700,서울특별시 광진구 구의동 243-31,24000
